In [1]:
import gbasis
import numpy as np
import chemtools
from numpy.testing import assert_allclose
from pyscf.dft import numint
from iodata import load_one
from functools import wraps
from pyscf import gto, scf, dft, df
from pyscf.tools import cubegen
from time import perf_counter
from gbasis.wrappers import from_iodata
from gbasis.evals.eval import evaluate_basis

def time_func(func, num_of_iter=100):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = perf_counter()
        # run the iters
        for _ in range(num_of_iter):
            func(*args, **kwargs)
        # end and print results
        end = perf_counter()
        result = end - start
        print(f'{num_of_iter} iters of {func.__name__} took {result} seconds')
        return func(*args, **kwargs)
    return wrapper

np.random.seed(123)

# Molecule setup

In [2]:
# get the mols
mol_chemtools = chemtools.Molecule.from_file('water_neutral.fchk')

mol_pyscf = gto.M(
    atom = 'O 0.01086593, 0.00805023, -0.00568808; H 0.52133765,  1.67452412,  0.47604091; H 1.13869228, -0.44555963, -1.34435117',
    basis = 'CC-pVTZ'
)
mf = scf.RHF(mol_pyscf).run()

# get the points
grid_3d = np.random.random((5,3))

# the two below return alpha oribital coefficients
print(mol_chemtools.mo.coefficient[0].shape) 
print(mf.mo_coeff.shape)

try:
    assert_allclose(mol_chemtools.mo.coefficient[0], mf.mo_coeff, rtol=1e-5, atol=1e-08)
except AssertionError as e:
    print(e)

converged SCF energy = -75.6682970590963
(58, 58)
(58, 58)

Not equal to tolerance rtol=1e-05, atol=1e-08

Mismatched elements: 3339 / 3364 (99.3%)
Max absolute difference: 7.31570753
Max relative difference: 1.2576048e+16
 x: array([[ 9.732297e-01, -2.049642e-01, -8.922210e-08, ..., -9.718058e-08,
        -7.764849e-07,  1.526411e+00],
       [-9.379156e-03,  4.657901e-01,  2.837934e-07, ..., -1.798607e-06,...
 y: array([[ 9.766285e-01, -4.612257e-03, -3.234713e-16, ..., -4.343873e-01,
        -1.944477e-06,  2.528644e+00],
       [-3.630840e-04,  5.651166e-01,  1.405175e-16, ..., -4.871092e-01,...


# *** EVERYWHERE POSSIBLE INSTEAD OF PYSCF PARAMETERS (LIKE MO COEFFS OR DM) 
# CHEMTOOLS' EQUIVALENTS WERE PASSED ***

# Evaluation of MOs
### Problem: pyscf requires mo_coeff to calculate mo - need to get those from chemtools Molecule somehow (custom class?). 
### otherwise energy calculation is required

In [3]:
@time_func
def pyscf_calc():
    ao = numint.eval_ao(mol_pyscf, grid_3d)
    result = ao.dot(mol_chemtools.mo.coefficient[0])
    return result
pyscf_result = pyscf_calc()

@time_func
def gbasis_calc():
    result = mol_chemtools.compute_molecular_orbital(grid_3d)
    return result
gbasis_result = gbasis_calc()

print(f'pyscf output shape: {pyscf_result.shape}')
print(f'gbasis output shape: {gbasis_result.shape}')

try:
    assert_allclose(pyscf_result.T, gbasis_result)
except AssertionError as e:
    print(e)

100 iters of pyscf_calc took 0.08522437200008426 seconds
100 iters of gbasis_calc took 1.1743194870032312 seconds
pyscf output shape: (5, 58)
gbasis output shape: (58, 5)

Not equal to tolerance rtol=1e-07, atol=0

Mismatched elements: 290 / 290 (100%)
Max absolute difference: 2.30514026
Max relative difference: 119573.0610977
 x: array([[ 4.118575e-02,  8.980544e-03,  1.094219e-03,  1.661091e-02,
         1.491916e-01],
       [ 3.319246e-01,  2.522354e-01,  1.701611e-01,  2.528460e-01,...
 y: array([[-4.390640e-02, -8.800590e-02, -6.551161e-02, -8.953844e-02,
         1.408781e-01],
       [ 2.712307e-01,  2.393145e-01,  1.532566e-01,  2.815618e-01,...


# Gradients 

In [4]:
@time_func
def pyscf_grad_calc():
    result = numint.eval_ao(mol_pyscf, grid_3d, deriv=1)
    return result[1:4]
pyscf_grad = pyscf_grad_calc()

@time_func
def gbasis_grad_calc():
    result = mol_chemtools.compute_gradient(grid_3d)
    return result
gbasis_grad = gbasis_grad_calc()

print(f'pyscf output shape: {pyscf_grad.shape}')
print(f'gbasis output shape: {gbasis_grad.shape}')

100 iters of pyscf_grad_calc took 0.0705173400019703 seconds
100 iters of gbasis_grad_calc took 10.950103749000846 seconds
pyscf output shape: (3, 5, 58)
gbasis output shape: (5, 3)


### So if gradient is supposed to ouput dx, dy, and dz, does each row of the pyscf output correspond to same?

# Hessian

In [5]:
@time_func
def pyscf_hessian_calc():
    result = numint.eval_ao(mol_pyscf, grid_3d, deriv=2)
    # index 0 is the ao value
    # indices 1:4 are for x, y, z 
    # indices 4:10 are for xx xy xz yy yz zz 
    return result[4:10]
pyscf_hessian = pyscf_hessian_calc()

@time_func
def gbasis_hessian_calc():
    result = mol_chemtools.compute_hessian(grid_3d)
    return result
gbasis_hessian = gbasis_hessian_calc()

print(f'pyscf output shape: {pyscf_hessian.shape}')
print(f'gbasis output shape: {gbasis_hessian.shape}')

100 iters of pyscf_hessian_calc took 0.05059200499817962 seconds
100 iters of gbasis_hessian_calc took 52.76469741400069 seconds
pyscf output shape: (6, 5, 58)
gbasis output shape: (5, 3, 3)


### Same problem as above, except we now have more dimensions

# Laplacian

### In theory, getting the laplacian would require just taking the trace of the hessian
### But our matrices are not equivalent, so it's an impasse

# ESP
## GBasis ESP function doesn't support mixed coordinated - can't test

In [6]:
@time_func
def pyscf_esp_calc(mol, points=grid_3d):
    # Nuclear potential at given points
    Vnuc = 0
    for i in range(mol.natm):
        r = mol.atom_coord(i)
        Z = mol.atom_charge(i)
        rp = r - points
        Vnuc += Z / np.einsum('xi,xi->x', rp, rp)**.5
    # Potential of electron density
    fakemol = gto.fakemol_for_charges(points)
    Vele = np.einsum('ijp,ij->p', df.incore.aux_e2(mol, fakemol), mf.make_rdm1())

    MEP = Vnuc - Vele     # MEP at each point
    return MEP
pyscf_esp = pyscf_esp_calc(mol_pyscf)

@time_func
def gbasis_esp_calc():
    mo = mol_chemtools.compute_esp(grid_3d)
    return mo
gbasis_hessian = gbasis_esp_calc()

100 iters of pyscf_esp_calc took 0.1352815130012459 seconds


NotImplementedError: Mixed coordinate types are not supported yet.

# Density
## Same as above - mixed coordinates for gbasis implementation are not supported yet
## Additional question - is density calculation in PySCF done with respect to AO or MO?

In [7]:

@time_func
def pyscf_dens_calc():
    dm = mol_chemtools.density_matrix
    ao_value = numint.eval_ao(mol_pyscf, grid_3d, deriv=1) 

    # compare calcs using coeffs from chemtools and pyscf
    mo_chemtools = ao_value.dot(mol_chemtools.mo.coefficient[0])
    # The first row of rho is electron density, the rest three rows are electron
    # density gradients which are needed for GGA functional
    density_chemtools = numint.eval_rho(mol_pyscf, mo_chemtools, dm, xctype='GGA') 

    return density_chemtools[0]
dens_pyscf = pyscf_dens_calc()


@time_func
def gbasis_density_calc():
    result = mol_chemtools.compute_density(grid_3d)
    return result
dens_gbasis = gbasis_density_calc()

try:
    assert_allclose(dens_pyscf, dens_gbasis)
except AssertionError as e:
    print(e)

100 iters of pyscf_dens_calc took 0.09980899299989687 seconds
100 iters of gbasis_density_calc took 1.1071392470003047 seconds

Not equal to tolerance rtol=1e-07, atol=0

Mismatched elements: 5 / 5 (100%)
Max absolute difference: 0.25112984
Max relative difference: 0.61904289
 x: array([0.143352, 0.098812, 0.059745, 0.410983, 0.452286])
 y: array([0.373131, 0.259378, 0.139309, 0.376618, 0.703416])


# KED

In [8]:
@time_func
def pyscf_ked_calc():
    dm = mol_chemtools.density_matrix
    ao_value = numint.eval_ao(mol_pyscf, grid_3d, deriv=1) 
    # The first row of rho is electron density, the rest three rows are electron
    # density gradients which are needed for GGA functional
    rho = numint.eval_rho(mol_pyscf, ao_value, dm, xctype='GGA') 
    return rho[0]
pyscf_ked = pyscf_ked_calc()

@time_func
def gbasis_ked_calc():
    result = mol_chemtools.compute_ked(grid_3d)
    return result
gbasis_ked = gbasis_ked_calc()

print(f'pyscf output shape: {pyscf_ked.shape}')
print(f'gbasis output shape: {gbasis_ked.shape}')
assert_allclose(pyscf_ked, gbasis_ked)

100 iters of pyscf_ked_calc took 0.1600142009992851 seconds
100 iters of gbasis_ked_calc took 3.5803151840009377 seconds
pyscf output shape: (5,)
gbasis output shape: (5,)


AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

Mismatched elements: 5 / 5 (100%)
Max absolute difference: 4.86137026
Max relative difference: 0.84371824
 x: array([0.512645, 0.264598, 0.130788, 0.430418, 0.900471])
 y: array([0.947636, 0.353499, 0.166211, 0.564202, 5.761841])

# PySCF cubegen module - keeping these for now just in case

In [ ]:
# pyscf has only 3 properties that can be outputed in .cube format 

def pyscf_dens(dim_xyz=100):
    outfile = 'out'
    cubegen.density(mol_pyscf, outfile, dm, dim_xyz, dim_xyz, dim_xyz)
    subprocess.run(["rm", "out"])

def pyscf_esp(dim_xyz=100):
    outfile = 'out'
    cubegen.mep(mol_pyscf, outfile, dm, dim_xyz, dim_xyz, dim_xyz)
    subprocess.run(["rm", "out"])

def pyscf_orb(dim_xyz=100):
    outfile = 'out'
    cubegen.orbital(mol_pyscf, outfile, dm, dim_xyz, dim_xyz, dim_xyz)
    subprocess.run(["rm", "out"])
